In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

In [5]:
# Fetch historical stock data
def fetch_stock_data(symbol, start, end):
    stock = yf.Ticker(symbol)
    data = stock.history(start=start, end=end)
    data['Date'] = data.index
    return data

# Detect support levels for buying
def calculate_support_levels(data, recent_period=60, drop_factors=[0.02, 0.05, 0.1]):
    """
    Calculate three support levels based on recent lows, applying increasing drops for each level.
    """
    recent_low = data['Close'].tail(recent_period).min()
    levels = [recent_low * (1 - drop) for drop in drop_factors]
    return levels

# Calculate fund allocation for each level
def allocate_funds(total_funds=200000, levels=[0.02, 0.05, 0.1]):
    allocation_ratios = [0.2, 0.3, 0.5]  # Aggressive to Conservative ratios
    return [total_funds * ratio for ratio in allocation_ratios]

# Visualize with suggested limit orders and allocation
def visualize_limit_orders(data, support_levels, allocations):
    fig = go.Figure()
    
    # Plot the close price
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Close'], mode='lines', name='Close Price', line=dict(color='blue')))
    
    # Plot the suggested limit orders with annotations
    for i, (level, allocation) in enumerate(zip(support_levels, allocations)):
        label = f"Buy Level {i+1}: ${level:.2f} (Alloc: ${allocation:.2f})"
        fig.add_trace(go.Scatter(x=[data['Date'].iloc[-1] + pd.Timedelta(days=30)], y=[level], mode='markers+text', 
                                 name=label, text=label, textposition="bottom right",
                                 marker=dict(symbol='triangle-down', color='green', size=10)))
    
    fig.update_layout(
        title="Price Chart with Suggested Buy Levels and Fund Allocation",
        xaxis_title="Date",
        yaxis_title="Price",
        showlegend=True
    )
    
    fig.show()

# Main function to execute the analysis
def analyze_stock_with_limit_orders(symbol, total_funds=8000):
    # Define the time range for the analysis
    end_date = datetime.today()
    start_date = end_date - pd.DateOffset(years=5)
    
    # Fetch data and calculate support levels and allocations
    data = fetch_stock_data(symbol, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    support_levels = calculate_support_levels(data)
    allocations = allocate_funds(total_funds=total_funds)
    
    # Visualize results
    visualize_limit_orders(data, support_levels, allocations)
    print(f"Suggested buy levels and allocations for {symbol}:")
    for i, (level, allocation) in enumerate(zip(support_levels, allocations), 1):
        print(f"Level {i}: ${level:.2f}, Allocation: ${allocation:.2f}")

# Example usage
symbol = 'HCA'
analyze_stock_with_limit_orders(symbol)


Suggested buy levels and allocations for HCA:
Level 1: $341.77, Allocation: $1600.00
Level 2: $331.31, Allocation: $2400.00
Level 3: $313.88, Allocation: $4000.00
